In [1]:
%use dataframe, kandy

In [2]:
USE {
    dependencies {
        implementation("org.apache.commons:commons-math3:3.6.1")
    }
}

In [3]:
fun DataFrame<*>.addProjectColumn(): DataFrame<*> =
    add(this["benchmark"].map { (it as String).substringBefore('-') }.rename("project"))

val folder = "../output/base"

val smtData = DataFrame.readCSV("${folder}/smtData.csv").addProjectColumn()
val fullSmtData = DataFrame.read("${folder}/fullSmtData.csv").addProjectColumn()

println("smtData: ${smtData.columnNames()}")
println("fullSmtData: ${fullSmtData.columnNames()}")


smtData: [benchmark, cnt, sat, unsat, unknown, project]
fullSmtData: [benchmark, file, result, numberOfVariables, reps, numberOfClauses, height, ucNumberOfVariables, ucReps, ucNumberOfClauses, ucHeight, project]


In [4]:
import org.jetbrains.letsPlot.scale.guideColorbar
import org.jetbrains.letsPlot.scale.guideLegend

smtData.plot {
    points {
        x(sat) { axis.name = "SAT" }
        y(unsat) { axis.name = "UNSAT" }
        color(sat.values.zip(unsat.values).map { (s, u) -> s < u }) {
            scale = categorical(
                false to Color.GREEN, true to Color.RED
            )
            legend {
                breaksLabeled(true to "SAT < UNSAT", false to "UNSAT > SAT")
            }
        }
    }
    abLine {
        slope.constant(1)
        intercept.constant(0)
        color = Color.ORANGE
        type = LineType.DASHED
    }
    layout {
        size = 800 to 400
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="3Hj4Pl"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"color":[false,false,true,false,false,false,true,true,true,false,true,false,false,false,false,true,true,false,false,true,false,true,true,false,true,false,false,false,true,false,false,false,false,true,true,false,true,true,false,false,true,false,false,true,true,true,false,true,false,true,false,false,false,false],
"sat":[77.0,252.0,5.0,6.0,94.0,3.0,78.0,19.0,0.0,33.0,16.0,82.0,196.0,107.0,25.0,41.0,2.0,64.0,45.0,30.0,131.0,34.0,16.0,45.0,114.0,50.0,54.0,237.0,44.0,97.0,84.0,122.0,88.0,49.0,31.0,9.0,14.0,7.0,1.0,15.0,6.0,60.0,9.0,5.0,122.0,12.0,17.0,57.0,20.0,12.0,11.0,6.0,18.0,60.0],
"unsat":[21.0,73.0,6.0,2.0,40.0,2.0,185.0,50.0,98.0,8.0,17.0,75.0,50.0,2.0,7.0,53.0,111.0,15.0,40.0,45.0,63.0,309.0,24.0,2.0,124.0,9.0,1.0,236.0,72.0,14.0,47.0,83.0,60.0,69.0,45.0,8.0,22.0,11.0,0.0,8.0,11.0,10.0,0.0,12.0,1037.0,56.0,2.0,67.0,3.0,16.0,1.0,5.0,0.0,8.0]
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"SAT",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"UNSAT",
"limits":[null,null]
},{
"aesthetic":"color",
"breaks":[true,false],
"values":["#3ba272","#ee6666"],
"limits":[false,true],
"labels":["SAT < UNSAT","UNSAT > SAT"]
}],
"layers":[{
"mapping":{
"x":"sat",
"y":"unsat",
"color":"color"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"point",
"data":{
}
},{
"mapping":{
},
"stat":"identity",
"color":"#fc8452",
"intercept":0.0,
"linetype":"dashed",
"sampling":"none",
"position":"identity",
"geom":"abline",
"slope":1.0,
"data":{
}
}]
};
 var plotContainer = document.getElementById("3Hj4Pl");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 250 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 UNSAT 
 
 
 
 
 SAT 
 
 
 
 
 
 
 
 
 color 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 UNSAT > SAT 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 SAT < UNSAT

In [5]:
smtData.plot { 
    val transform: (Number) -> Double = {
        log(it.toDouble() + 1, 5.0)
    }
    val s = sat.map { transform(it) }
    val u = unsat.map { transform(it) }
    
    points { 
        x(s) {
            axis {
                name = "SAT"
                breaksLabeled(0.0 to "0", 1.0 to "4", 2.0 to "24", 3.0 to "124")
            }
        }
        y(u) { 
            axis {
                name = "UNSAT"
                breaksLabeled(0.0 to "0", 1.0 to "4", 2.0 to "24", 3.0 to "124", 4.0 to "624")
            }
        }
        color(sat.values.zip(unsat.values).map { (s, u) -> s < u }) {
            scale = categorical(
                false to Color.GREEN, true to Color.RED
            )
            legend {
                breaksLabeled(true to "SAT < UNSAT", false to "UNSAT > SAT")
            }
        }
    }
    abLine {
        slope.constant(1)
        intercept.constant(0)
        color = Color.ORANGE
        type = LineType.DASHED
    }
    layout {
        size = 800 to 400
    }
//    facetWrap(2) {
//        facet(project)
//    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ca83bJ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"color":[false,false,true,false,false,false,true,true,true,false,true,false,false,false,false,true,true,false,false,true,false,true,true,false,true,false,false,false,true,false,false,false,false,true,true,false,true,true,false,false,true,false,false,true,true,true,false,true,false,true,false,false,false,false],
"sat":[2.7069753937264607,3.438088195871358,1.1132827525593785,1.2090619551221675,2.8294828004351507,0.8613531161467861,2.7148905954742335,1.8613531161467862,0.0,2.1910509857959815,1.7603744277225881,2.7455800398746546,3.2826390430605152,2.9091716996047423,2.0243691992404753,2.322344707681546,0.6826061944859854,2.5936926411670824,2.378868651539773,2.133656214977323,3.0338554130377493,2.2090619551221677,1.7603744277225881,2.378868651539773,2.9481920934663797,2.4429806222085735,2.489896102404978,3.4001129409181488,2.3652123889719707,2.8488004683177284,2.760374427722588,2.9899782515341085,2.7889465850494126,2.4306765580733933,2.1533827903669653,1.4306765580733933,1.6826061944859854,1.2920296742201791,0.43067655807339306,1.7227062322935722,1.2090619551221675,2.554229543378943,1.4306765580733933,1.1132827525593785,2.9899782515341085,1.5936926411670824,1.7958889470453636,2.522895092240893,1.8916681496081529,1.5936926411670824,1.5439593106327716,1.2090619551221675,1.8294828004351504,2.554229543378943],
"unsat":[1.9205726604783713,2.6742660030263226,1.2090619551221675,0.6826061944859854,2.3073720570481235,0.6826061944859854,3.2469389675367015,2.4429806222085735,2.855108491376949,1.3652123889719707,1.7958889470453636,2.690835916581937,2.4429806222085735,0.6826061944859854,1.2920296742201791,2.478495141531349,2.9317681874157397,1.7227062322935722,2.3073720570481235,2.378868651539773,2.5840593484403582,3.564332773050716,2.0,0.6826061944859854,3.0000000000000004,1.4306765580733933,0.43067655807339306,3.397496789960219,2.665812336096604,1.6826061944859854,2.405312426779558,2.753021265754939,2.554229543378943,2.639738513195561,2.378868651539773,1.3652123889719707,1.9481920934663797,1.5439593106327716,0.0,1.3652123889719707,1.5439593106327716,1.4898961024049782,0.0,1.5936926411670824,4.315202848193254,2.512088994921136,0.6826061944859854,2.6217275438693743,0.8613531161467861,1.7603744277225881,0.43067655807339306,1.1132827525593785,0.0,1.3652123889719707]
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"breaks":[0.0,1.0,2.0,3.0],
"name":"SAT",
"limits":[null,null],
"labels":["0","4","24","124"]
},{
"aesthetic":"y",
"breaks":[0.0,1.0,2.0,3.0,4.0],
"name":"UNSAT",
"limits":[null,null],
"labels":["0","4","24","124","624"]
},{
"aesthetic":"color",
"breaks":[true,false],
"values":["#3ba272","#ee6666"],
"limits":[false,true],
"labels":["SAT < UNSAT","UNSAT > SAT"]
}],
"layers":[{
"mapping":{
"x":"sat",
"y":"unsat",
"color":"color"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"point",
"data":{
}
},{
"mapping":{
},
"stat":"identity",
"color":"#fc8452",
"intercept":0.0,
"linetype":"dashed",
"sampling":"none",
"position":"identity",
"geom":"abline",
"slope":1.0,
"data":{
}
}]
};
 var plotContainer = document.getElementById("ca83bJ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 24 
 
 
 
 
 
 
 
 
 124 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 24 
 
 
 
 
 
 
 124 
 
 
 
 
 
 
 624 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


In [6]:
fun DataFrame<*>.filterUnsat(): DataFrame<*> = filter { get("result") == "UNSAT" }

val unsatSmtData = fullSmtData.filterUnsat()

In [7]:
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalContinuousScale

fullSmtData.plot {
    boxplot(project, numberOfVariables)
    boxplot(numberOfVariables, xName = "ALL")
    y {
        axis.expand(additive = 0.5)
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
    }
    layout {
        title = "Variables distribution"
        size = 800 to 400
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="C4hc4W"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Variables distribution"
},
"mapping":{
},
"data":{
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"project",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"numberOfVariables",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"ALL",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"numberOfVariables",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"expand":[0.0,0.5],
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"y":"y"
},
"stat":"identity",
"data":{
"middle":[41.0,57.0,67.5,175.0],
"min":[0.0,0.0,4.0,0.0],
"max":[334.0,530.0,214.0,617.0],
"upper":[142.0,225.5,136.75,305.0],
"lower":[13.0,22.0,29.5,94.0],
"x":["FASTJSON","GUAVA","SEATA","SPOON"]
},
"sampling":"none",
"position":"identity",
"geom":"boxplot"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA",

In [8]:
unsatSmtData.plot {
    boxplot(project, ucNumberOfVariables)
    boxplot(ucNumberOfVariables, xName = "ALL")
    y {
        axis.expand(additive = 0.5)
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
    }
    layout {
        title = "Variables distribution in unsat cores"
        size = 800 to 400
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="SKuQdC"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Variables distribution in unsat cores"
},
"mapping":{
},
"data":{
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"project",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"ucNumberOfVariables",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"ALL",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"ucNumberOfVariables",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"expand":[0.0,0.5],
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"y":"y"
},
"stat":"identity",
"data":{
"middle":[5.0,6.0,8.0,4.0],
"min":[0.0,0.0,2.0,4.0],
"max":[14.0,75.0,15.0,4.0],
"lower":[3.0,3.0,6.0,4.0],
"upper":[8.0,32.5,10.0,4.0],
"x":["FASTJSON","GUAVA","SEATA","SPOON"]
},
"sampling":"none",
"position":"identity",
"geom":"boxplot"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","GUAVA","SEATA","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON","SPOON

In [9]:
unsatSmtData.plot {
    boxplot(project, ucNumberOfClauses)
    boxplot(ucNumberOfClauses, xName = "ALL")
    y {
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
        axis.expand(additive = 0.5)
    }
    layout {
        title = "Clauses distribution in unsat cores"
        size = 800 to 400
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="W2FRhK"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Clauses distribution in unsat cores"
},
"mapping":{
},
"data":{
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"project",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"ucNumberOfClauses",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"ALL",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"ucNumberOfClauses",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"expand":[0.0,0.5],
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"y":"y"
},
"stat":"identity",
"data":{
"middle":[5.0,7.0,8.0,5.0],
"min":[4.0,1.0,3.0,5.0],
"max":[7.0,80.0,10.0,5.0],
"lower":[5.0,4.0,6.0,5.0],
"upper":[6.0,34.5,8.0,5.0],
"x":["FASTJSON","GUAVA","SEATA","SPOON"]
},
"sampling":"none",
"position":"identity",
"geom":"boxplot"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FA

In [10]:
fullSmtData
//    .filter { project == "GUAVA" }
    .plot {
        statDensity(numberOfVariables) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            title = "Variables distribution"
            size = 800 to 400
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="2D7hL5"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Variables distribution"
},
"mapping":{
},
"data":{
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density"
},
"stat":"identity",
"data":{
"density":[0.18736859999221475,0.4175423035118008,0.5477081046007676,0.6243132958146417,0.6770201289322269,0.7246890438595258,0.7728534519277389,0.8174830144475603,0.8444691235244494,0.8639406212528665,0.8793390318000939,0.8903723368110067,0.8993793498040988,0.9067557017842064,0.9116752237572534,0.9167869191786964,0.9212277471961218,0.9247581485463845,0.9279645257720862,0.9299637479632771,0.9318573885743096,0.9334131302411768,0.9350206718666979,0.9364867222654252,0.9381908910684054,0.9402058933907206,0.9414889260970721,0.9426318402990482,0.9442952040303184,0.9462524493261264,0.9482338664289395,0.9495577244683571,0.950676885455569,0.952117469924643,0.9533416313850658,0.9544016119770323,0.955706696284779,0.9568939947420385,0.9578916318971878,0.9584263091587156,0.9591286020693895,0.9598391957646125,0.9606655343935129,0.9623113180592215,0.9641486342144943,0.9652728101879775,0.9659288985419346,0.966614578052528,0.9674850288679893,0.9683349611536552,0.969138746041298,0.9700821087963253,0.9708453988084822,0.9712382793139098,0.971476455873054,0.971624588639922,0.9718883587477815,0.9722879166406816,0.9727670723947909,0.9730608916037631,0.9733766461300014,0.9736847695640991,0.974150319327512,0.974651436985639,0.9751639821185705,0.9757192223434359,0.9762724937476999,0.9765400395535145,0.9768499811219348,0.9770770863420555,0.9773042221664386,0.9776437463183443,0.9778773772317495,0.977978660510739,0.9784068797037472,0.9788904990878247,0.9790944289865051,0.9795667819323178,0.9797449386924645,0.9799661534731763,0.9802789929801201,0.9804216732281333,0.9806398536806779,0.9808966658641018,0.9810884306761504,0.9813179047943388,0.9814636661827164,0.981659322144922,0.98197902158251,0.9823409636028121,0.9825924022331177,0.9829934787563214,0.9833082529825699,0.9835743513557862,0.9838281495463378,0.9840195952894373,0.9840378162173472,0.9840712769242981,0.9842686759113405,0.9846579301100334,0.9850039669842622,0.9853263355550059,0.985661422629294,0.9860161926562668,0.986175694852231,0.9861961821423874,0.9862959836216159,0.9866458095247901,0.9871331487134443,0.9872621023053529,0.9872661681910649,0.9872849851507849,0.987469039083607,0.9877631397600491,0.9880514645603763,0.9882740260250327,0.988340173285724,0.988342026664654,0.9883446770422694,0.9884304001194081,0.988634442727841,0.9887692020318145,0.9889297861143886,0.9891953696176208,0.9893923485183171,0.9894758471338703,0.9896628684050129,0.9897744181948123,0.9899766073140714,0.9900375924009194,0.9901419550658008,0.9903778174152315,0.990486828678277,0.9904998920852698,0.9906511311085399,0.9910567702750933,0.9913412755870118,0.9916208241673876,0.991767620718982,0.9919657622744207,0.9920310949746467,0.9920332949368822,0.992059703854388,0.9921604605202038,0.9922027143469493,0.9923056924104794,0.992420199179967,0.9924898979968844,0.992498531238896,0.9925691866613304,0.9926883276498931,0.9927916622066755,0.9928766865671809,0.9930306856537348,0.9931045872158054,0.9931215930616805,0.9932149300980003,0.993260834938503,0.9932672355198439,0.993340288629663,0.9934546188460308,0.9935559738954522,0.9936093308034696,0.9937087985416116,0.9937827347600224,0.9940145229328265,0.9943610765210575,0.9945067225044147,0.9946136300932424,0.994645235227817,0.9946459133362183,0.994645914326117,0

In [11]:
fullSmtData
//    .filter { result == "UNSAT" }
//    .filter { project == "GUAVA" }
    .plot {
        statDensity(numberOfClauses) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            title = "Clauses distribution"
            size = 800 to 400
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="6jlQVI"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Clauses distribution"
},
"mapping":{
},
"data":{
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density"
},
"stat":"identity",
"data":{
"density":[0.1672770611389078,0.36549496538711473,0.4904549836700333,0.5710162085098338,0.6320147484200825,0.6748112416408881,0.7065827398919394,0.7402737343054135,0.7769762442072344,0.8095922219559497,0.835592711278345,0.8587682737701704,0.8783419532561031,0.8923108276632481,0.9034093716316044,0.9124828135051785,0.919849719717828,0.9250861642217305,0.9289773071694735,0.9325562650192774,0.9361212841793025,0.9390743495088797,0.9408988151886982,0.9427586656762293,0.9451286309856225,0.9470693069201842,0.9489669775126168,0.9513666542826579,0.9539810076455021,0.9561299899462522,0.9575689101135224,0.9589058001192611,0.9602207899169436,0.9611881585458878,0.9621986338864932,0.9633764380673744,0.9644521684893234,0.9655301584002631,0.9666619946060085,0.9676780446739287,0.968582218362084,0.9693670740619088,0.9699467178479612,0.9704116153982831,0.9708349027570697,0.9712039442174709,0.9716668776666872,0.9723247214282682,0.9733403817984737,0.974072607793645,0.9744664609095564,0.9749089488745553,0.9754162477620629,0.9757967143947522,0.9761659124417407,0.9765715639531455,0.9770385511517776,0.9775048528546161,0.9777927802044892,0.9779367112823901,0.9780262275243277,0.9781853823278378,0.9785387798153614,0.9787926199689578,0.9790427805823552,0.9796852312606843,0.9804969060556651,0.9810239161901118,0.9811581405940972,0.9811984020351299,0.9813641647332928,0.9815776707607586,0.981701672980593,0.9818789760822731,0.9820978741710976,0.9823702416379756,0.9826517862819516,0.9828443915797951,0.9831135439613278,0.9833282709776303,0.9834738709706597,0.9835809529888185,0.983685884827209,0.9837933046554993,0.9839423863053056,0.9842149283060292,0.9844800441404727,0.9846329234353988,0.9847879911162496,0.984994134316008,0.9851882965607749,0.9855825303843789,0.986131545659511,0.9866794698284185,0.9871095008010293,0.9872799618791641,0.9873062377734875,0.9873836069372954,0.9876350208776381,0.9878891140901638,0.9880658374073368,0.9882790290861123,0.9883925423333763,0.988504575802605,0.9887868444197431,0.989007433108392,0.9891379086872585,0.9893829112102183,0.9896757740975318,0.9898466695114638,0.9899452583596786,0.9900044242142872,0.9901749111982142,0.9905316720435884,0.9909552756135447,0.9912859369476085,0.991485640879687,0.9915654712119559,0.9916992742198257,0.9919211332347937,0.9920951727769836,0.9922494754792663,0.992322869049354,0.9923376463869186,0.9923845736107266,0.9924817295946904,0.9925814822114071,0.9926442863125036,0.9927031959201017,0.9928183519717112,0.9929261854435559,0.9929971004238187,0.9931073610564414,0.9932396887556463,0.9933695719287856,0.9935020339174926,0.9936243376695135,0.9937176212887553,0.9937459293565238,0.9937493092566353,0.9937726108021392,0.9938487392227183,0.9938985005686325,0.9939050004711928,0.9939051701357012,0.993905171020704,0.993905171020704,0.9939051720344313,0.9939053594700848,0.9939122849959297,0.9939634205225324,0.9940388710256922,0.9940611180012173,0.9940624288398817,0.994062480801877,0.9940648456630318,0.9940954415240032,0.9941745438290669,0.994215411983763,0.9942196313803499,0.9942197184336211,0.9942197213562405,0.9942200910402302,0.9942309212078693,0.9943002596508685,0.9944260959359177,0.9945141971649591,0.99453570933787,0.9945672538836512,0.9946468401136188,0.994703602916053,0.99481547032

In [12]:
unsatSmtData
//    .filter { project == "GUAVA" }
    .plot {
        statDensity(ucNumberOfVariables) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            title = "Variables distribution in unsat cores"
            size = 800 to 400
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="H4LK3i"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Variables distribution in unsat cores"
},
"mapping":{
},
"data":{
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density"
},
"stat":"identity",
"data":{
"density":[0.034813943232298354,0.1690057971663136,0.5498230860450087,0.7042300930681835,0.7940126774949834,0.8363465470342938,0.8504419859941864,0.8593933994339011,0.8895756714178964,0.8960724215419364,0.9006860580803048,0.9073483811944284,0.9099728236720845,0.9103982266173706,0.9136298168203203,0.9190977357442406,0.9211164821482837,0.9219520750407539,0.922836370584429,0.923564818077647,0.925044826719886,0.9254282343281406,0.9258117037605328,0.9261667241033783,0.9268765110959799,0.9272782589115272,0.9278310344627789,0.9282381866030552,0.9292091287352897,0.9296503353737089,0.930255672605074,0.9314628468361472,0.9317381712102538,0.9325015732388191,0.9327946552282698,0.9333336906978981,0.9341378846970848,0.9346918662475732,0.9352567293896819,0.9358124939173476,0.9363033025010266,0.9369047360487721,0.9378171786969981,0.9383924201443788,0.9391634747633613,0.9407635908737241,0.9411671524797341,0.9415893772040751,0.9416318538074726,0.9428268292689599,0.9430259187042865,0.9435677601947074,0.9441144378633282,0.9452100022287124,0.9455191124420919,0.9458686605158255,0.9459266583289039,0.9469549212827157,0.9473985270815461,0.9481653161737307,0.9486757897711005,0.9493382418981184,0.9495303052128361,0.9500761408958649,0.9506215825351964,0.9508679053617661,0.9513671865021224,0.9518631021157933,0.952497532175029,0.9530177705992173,0.953425841744548,0.9538341730507224,0.9539073439761468,0.9543850231201749,0.9544511629948393,0.9545272054628707,0.9546462236197198,0.9548131968917863,0.9548668960508897,0.9550092705925941,0.9551049988716006,0.9555507707349105,0.955699147440576,0.955849052611339,0.956355069897372,0.9571349732355519,0.9571718777138399,0.9573325498898482,0.9574965309421174,0.9580099348738833,0.9580115805865391,0.9584041315825566,0.9585605922265941,0.9592985485555103,0.9594688454988404,0.9598165326797722,0.9598191611186672,0.9598191877147147,0.9601509003830346,0.9601697947405818,0.9604006337851944,0.9611311291063461,0.9614567477559043,0.9617056034322351,0.962159346125555,0.962449730350225,0.962465145261265,0.9629980024544096,0.9632571392736456,0.9635301031280493,0.9638102803081537,0.9640952470446585,0.9646510032963408,0.9651360366421546,0.9654524365521432,0.9659808163612247,0.9662953394540992,0.9662953568234951,0.9665033078483927,0.966723213956945,0.9670682258439796,0.9673999622489388,0.9673999888450026,0.9674052457228057,0.9681030406522155,0.9687874328944365,0.968837220592005,0.9695635984849944,0.9699304481351774,0.9699320938443408,0.9704454982143573,0.9707425209509923,0.9709359156787803,0.9709705820419764,0.9709705823031869,0.97108394089625,0.9711249946293308,0.9711249946413778,0.9711257067840482,0.9715276845216518,0.971527868467379,0.9716152586918441,0.9716722970925111,0.9720801197571769,0.9724888560700163,0.9729708602357566,0.9738574748210661,0.9738585131403205,0.9742665701635036,0.9744455429864388,0.9746960028562041,0.9746964099158103,0.975503859082097,0.975505157667459,0.9755053075727524,0.9759465292492822,0.9764513109967436,0.9764910764882679,0.9766091825945695,0.9766091828856266,0.9766091828856266,0.9766091828856266,0.9766091828856266,0.9766091828856266,0.9766091828886093,0.976633506475141,0.9771512532864276,0.9778589742215291,0.978559187768391,0.9785833359125409,0.97876677373885

In [13]:
unsatSmtData
//    .filter { project == "GUAVA" }
    .plot {
        statDensity(ucNumberOfClauses) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            title = "Clauses distribution in unsat cores"
            size = 800 to 400
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="MzNZ2A"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Clauses distribution in unsat cores"
},
"mapping":{
},
"data":{
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density"
},
"stat":"identity",
"data":{
"density":[0.04333805714434474,0.1858129734025653,0.7063542914781372,0.7940165474329746,0.8480700798957657,0.871299462584638,0.8785944761624818,0.8957387451091017,0.9050695221786209,0.9110393125720108,0.9174146001472282,0.9181340272425794,0.9193236523900318,0.9219853068430619,0.9235948564718769,0.9251522202126441,0.9261924668428128,0.9267962999862539,0.9268479141221105,0.9279421702157868,0.9280199740693844,0.9285702399676824,0.9301315851285163,0.9301477246906449,0.9312154985440533,0.932285616598376,0.9323306582200508,0.933413945074286,0.9339479665942846,0.9345125478519387,0.9345420093114285,0.935610004368297,0.935638700552131,0.9361726613990933,0.937222077419038,0.9377369694908501,0.9378032827325286,0.938330381876715,0.939863631191065,0.9399110324778497,0.9409563131803018,0.9415376013991832,0.9415627383197905,0.9426292241231731,0.9426536923259506,0.9432234355658581,0.9437429859056029,0.9448084627773943,0.9448316471414524,0.9453640930091607,0.9459076586383331,0.9464116176033408,0.9469832891359385,0.9474965379695197,0.9480281185882284,0.9491013989759638,0.9491637141326228,0.9497049304420476,0.9502457838846915,0.9502557765372605,0.9518032655405307,0.9523088579730082,0.9523532137030165,0.9533769417951745,0.9534315673835089,0.9539697910626934,0.9545076191828884,0.955045046784902,0.955534753156662,0.9556303962960021,0.9561268280176282,0.9571326590229646,0.9577173788650926,0.9577836791638477,0.9577917745634548,0.9583178561064007,0.9583257260485462,0.9587990980233311,0.9587991087845223,0.9593238486823492,0.9593312839191188,0.9598010492067203,0.9598558890010662,0.9598630239970675,0.9607969432581451,0.9608535121941103,0.9609174894259181,0.9614460290476972,0.9619157464217101,0.9623778570991289,0.9628389681628148,0.963299056492612,0.9633598229480549,0.9633659745740536,0.9634899913355316,0.9635019704700074,0.9644750632908572,0.9644808808519628,0.9649336613789059,0.9649336615053212,0.9649336615053296,0.9650001875093593,0.9654540116982273,0.9659691570295457,0.9660428978824832,0.9661172580301928,0.9667060010445538,0.9667160237988476,0.967157656211458,0.9672296463282054,0.9681130442235739,0.9681130665125596,0.9686241140063476,0.9690644731396,0.9695745553875381,0.969655253092702,0.9701687512777005,0.970250789302882,0.9707631982147795,0.9707673968389163,0.9711946729802815,0.9716206779905209,0.9717021881894483,0.9721296092752465,0.972129609338916,0.9721296393690738,0.9726339175222605,0.9728084065871581,0.9732328190430845,0.9737356512342409,0.9741536906798087,0.9741536907322587,0.9743325270259411,0.974429528204126,0.974842127864685,0.9752500508294486,0.9752500508750407,0.9753436374347377,0.9753467466133645,0.9753467466133645,0.9753467880902799,0.9757466173926579,0.9757466607151772,0.9761437510947585,0.9761437511311789,0.9762443346072384,0.9766400202995517,0.9767424460778286,0.9776286529893414,0.9776312650240905,0.9777363749961443,0.9782308836650944,0.9786179077580787,0.9786179624177572,0.979108433883904,0.9791108061245348,0.9791109224912191,0.9798657457981981,0.9799783843006977,0.9803551440562605,0.9803551440793523,0.9803551440793523,0.9803551440793523,0.9803551440793523,0.9803551440793523,0.9803551440793523,0.9803552157435608,0.9807181155576172,0.9807181902686184,0.9813252075974237,0.9818119249337627,

In [14]:
unsatSmtData
    .filter { ucNumberOfVariables < 15 }
    .plot {
        statDensity(ucReps) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            size = 800 to 400
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="jLtPGQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density"
},
"stat":"identity",
"data":{
"density":[0.009331559636437711,0.01626160644650763,0.019100061128524658,0.019741258341363786,0.019821143325159083,0.019826632442836393,0.01982684046221553,0.019826844810009087,0.019826844860127653,0.019826844860487355,0.01982684487169608,0.019826846558547235,0.019826986583154058,0.019833397141031817,0.019995261470943944,0.022249346768365267,0.03956159231638412,0.11289473967650948,0.28421594082840973,0.504958341280117,0.6618226990617841,0.7233016819082649,0.7365908595073083,0.7381804058643714,0.7383817671529569,0.7393688404205396,0.7448551908552573,0.7617376120298457,0.7903896632374448,0.817208540482088,0.831053424900942,0.8349953012042035,0.8356147046380276,0.8356784803814686,0.8358165466052412,0.8368124718233616,0.8408489343430996,0.8498722923295937,0.8609972769717205,0.8685620258866524,0.8713989881643655,0.8719858004063913,0.8720537198738044,0.8720751840254094,0.8722423420940586,0.873134001001404,0.8757594955610021,0.880023215885902,0.8838420444838385,0.88572845193145,0.8862423878505818,0.886319822537233,0.8863311526276155,0.8863943154873893,0.8868364450467349,0.8885512014050914,0.8922191762367573,0.8965464479159658,0.899362014761664,0.9003723871790215,0.9005723675338837,0.9005945671680569,0.9006021311303847,0.9006600665063917,0.9009558161766668,0.901789100881207,0.903083975089229,0.9041937262792912,0.9047182770607061,0.9048550240081342,0.9048747398247624,0.9048775142015073,0.9048924105230068,0.9049921912369983,0.9053624933105497,0.9061204364947465,0.9069760567355709,0.9075087648561426,0.9076916855426665,0.9077263578902096,0.9077308732661895,0.9077454993278211,0.9078732948338853,0.9084979727983612,0.9101821314424636,0.9126863587985073,0.9147400180108811,0.9156688712017319,0.9159006048250499,0.9159337070741764,0.9159620651064443,0.9162649998753815,0.9182149068645709,0.9251393783173399,0.9387013784220744,0.9533509432486692,0.9620784374538283,0.9649460359328614,0.9654657068844297,0.9655182295677889,0.9655301164171951,0.9656067381318801,0.965964735369092,0.9668882874397041,0.9682023278453169,0.9692334732821799,0.9696797406822177,0.9697862730326012,0.9698007245506671,0.9698104251887589,0.9699074604974026,0.9705050650655492,0.9725357525436834,0.976341472145485,0.9802751097787539,0.9825175236835542,0.9832225430869361,0.9833447934692916,0.9833564847811975,0.9833571014331803,0.9833571193714639,0.9833571196592603,0.9833571196618068,0.9833571196618224,0.9833571196624897,0.983357119739589,0.9833571246522513,0.9833572972948209,0.9833606434240906,0.9833964119796781,0.9836072864258817,0.9842929488847931,0.9855225398206148,0.9867386552501997,0.9874020206448944,0.987601590831962,0.9876347248466388,0.9876382878456831,0.9876459427559446,0.9877042035423226,0.9879537075338668,0.9885430585338087,0.9893108345279461,0.9898624785269877,0.9900810770039248,0.9901288518426571,0.9901346104198411,0.9901349932401937,0.9901350072760434,0.9901350075598656,0.990135007563031,0.9901350075630505,0.9901350075630505,0.9901350075630505,0.9901350075630505,0.9901350075630505,0.9901350075630505,0.9901350075630505,0.9901350075630505,0.9901350075630505,0.9901350075630505,0.9901350075630541,0.9901350075637543,0.9901350076378215,0.9901350119590405,0.9901351510024506,0.9901376185094823,0.9901617692388864,0.9902921357278769,0.9906802553064937,0.991317533411255,0.9918946403662505,0.9921

In [15]:
unsatSmtData
//    .filter { project == "FASTJSON" }
    .plot {
        points {
            x(numberOfClauses)
            y(ucNumberOfVariables)
        }
//    x {
//        scale = PositionalContinuousScale<Double>(
//            null, null, null, Transformation.LOG2
//        )
//    }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="JGN8Xe"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"ucNumberOfVariables":[2.0,3.0,3.0,0.0,3.0,22.0,7.0,2.0,0.0,35.0,5.0,3.0,2.0,2.0,2.0,2.0,5.0,0.0,5.0,2.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,5.0,7.0,0.0,5.0,6.0,3.0,0.0,2.0,6.0,0.0,2.0,0.0,6.0,0.0,0.0,6.0,10.0,2.0,3.0,6.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,7.0,8.0,2.0,7.0,8.0,8.0,2.0,4.0,2.0,6.0,3.0,2.0,3.0,6.0,6.0,6.0,5.0,6.0,6.0,0.0,3.0,7.0,6.0,2.0,6.0,7.0,3.0,8.0,13.0,6.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,5.0,3.0,2.0,10.0,5.0,5.0,0.0,2.0,31.0,5.0,2.0,4.0,31.0,4.0,5.0,5.0,5.0,34.0,3.0,3.0,4.0,4.0,6.0,6.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,6.0,4.0,56.0,2.0,23.0,0.0,2.0,5.0,5.0,6.0,6.0,6.0,3.0,6.0,9.0,8.0,6.0,8.0,3.0,9.0,9.0,9.0,6.0,6.0,8.0,8.0,11.0,8.0,8.0,11.0,8.0,3.0,4.0,4.0,8.0,29.0,5.0,9.0,11.0,12.0,9.0,3.0,8.0,8.0,30.0,28.0,8.0,7.0,8.0,4.0,12.0,31.0,3.0,28.0,8.0,3.0,3.0,4.0,6.0,8.0,3.0,3.0,3.0,3.0,6.0,3.0,6.0,8.0,11.0,3.0,9.0,8.0,5.0,8.0,4.0,30.0,0.0,4.0,4.0,29.0,8.0,8.0,5.0,8.0,11.0,5.0,5.0,30.0,4.0,24.0,8.0,3.0,4.0,9.0,8.0,2.0,8.0,8.0,6.0,6.0,30.0,8.0,9.0,6.0,29.0,9.0,0.0,9.0,8.0,9.0,8.0,8.0,8.0,9.0,3.0,9.0,8.0,7.0,8.0,9.0,8.0,6.0,8.0,3.0,9.0,6.0,8.0,10.0,9.0,8.0,8.0,11.0,3.0,8.0,8.0,6.0,5.0,6.0,8.0,9.0,8.0,10.0,9.0,11.0,3.0,6.0,5.0,8.0,22.0,7.0,7.0,9.0,8.0,3.0,11.0,6.0,9.0,3.0,8.0,11.0,8.0,8.0,8.0,7.0,8.0,8.0,9.0,8.0,8.0,6.0,3.0,9.0,7.0,9.0,11.0,8.0,8.0,8.0,9.0,8.0,9.0,9.0,9.0,4.0,30.0,4.0,8.0,9.0,30.0,4.0,6.0,8.0,30.0,9.0,6.0,8.0,8.0,2.0,4.0,2.0,5.0,2.0,5.0,2.0,3.0,4.0,5.0,3.0,4.0,2.0,2.0,2.0,2.0,5.0,3.0,2.0,2.0,12.0,2.0,5.0,5.0,5.0,2.0,5.0,5.0,4.0,2.0,2.0,4.0,2.0,5.0,2.0,2.0,5.0,4.0,2.0,2.0,5.0,2.0,2.0,5.0,2.0,2.0,2.0,5.0,5.0,5.0,8.0,6.0,7.0,10.0,29.0,30.0,9.0,7.0,6.0,7.0,4.0,6.0,7.0,2.0,7.0,6.0,10.0,8.0,7.0,10.0,4.0,7.0,8.0,7.0,6.0,6.0,7.0,4.0,3.0,6.0,6.0,30.0,4.0,4.0,9.0,7.0,17.0,4.0,3.0,9.0,10.0,9.0,10.0,17.0,6.0,10.0,10.0,10.0,7.0,8.0,29.0,14.0,7.0,7.0,8.0,7.0,6.0,11.0,10.0,6.0,6.0,9.0,7.0,30.0,11.0,4.0,7.0,11.0,6.0,8.0,6.0,6.0,9.0,8.0,11.0,6.0,2.0,10.0,8.0,6.0,7.0,8.0,7.0,10.0,7.0,6.0,11.0,6.0,7.0,7.0,30.0,11.0,29.0,3.0,7.0,10.0,7.0,7.0,29.0,11.0,4.0,6.0,3.0,4.0,3.0,3.0,6.0,3.0,6.0,8.0,3.0,6.0,2.0,3.0,2.0,3.0,6.0,4.0,2.0,2.0,10.0,6.0,6.0,5.0,2.0,3.0,2.0,2.0,5.0,3.0,5.0,3.0,3.0,2.0,2.0,2.0,3.0,5.0,5.0,4.0,0.0,5.0,3.0,8.0,0.0,2.0,3.0,5.0,0.0,9.0,5.0,5.0,5.0,2.0,0.0,5.0,5.0,5.0,6.0,10.0,5.0,0.0,4.0,5.0,8.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,6.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,3.0,0.0,3.0,5.0,4.0,3.0,8.0,3.0,10.0,4.0,4.0,8.0,8.0,4.0,8.0,4.0,3.0,8.0,9.0,8.0,2.0,4.0,8.0,10.0,5.0,5.0,6.0,6.0,6.0,10.0,28.0,4.0,8.0,4.0,4.0,5.0,8.0,8.0,4.0,8.0,8.0,2.0,10.0,10.0,2.0,4.0,9.0,8.0,3.0,8.0,3.0,3.0,8.0,8.0,4.0,8.0,3.0,2.0,2.0,2.0,6.0,6.0,6.0,4.0,6.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,3.0,5.0,3.0,5.0,5.0,3.0,5.0,3.0,5.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,5.0,2.0,5.0,5.0,2.0,3.0,5.0,5.0,3.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,7.0,5.0,2.0,2.0,5.0,3.0,0.0,0.0,5.0,5.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,2.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,5.0,5.0,5.0,0.0,3.0,0.0,5.0,5.0,5.0,5.0,5.0,0.0,2.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,2.0,5.0,0.0,5.0,5.0,0.0,5.0,5.0,2.0,0.0,5.0,5.0,5.0,0.0,0.0,0.0,5.0,0.0,5.0,5.0,2.0,5.0,5.0,0.0,5.0,5.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0,0.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,7.0,5.0,3.0,10.0,0.0,4.0,6.0,6.0,8.0,8.0,8.0,6.0,8.0,6.0,6.0,6.0,17.0,6.0,8.0,6.0,6.0,6.0,6.0,6.0,8.0,6.0,4.0,6.0,6.0,6.0,8.0,6.0,6.0,19.0,6.0,6.0,3.0,6.0,6.0,8.0,6.0,6.0,16.0,6.0,6.0,6.0,5.0,3.0,3.0,3.0,3.0,4.0,7.0,3.0,4.0,5.0,5.0,7.0,7.0,7.0,7.0,7.0,8.0,5

In [16]:
fullSmtData.plot {
    boxplot(result, numberOfClauses)
    y {
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="iXTiBR"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"result",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"numberOfClauses",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"y":"y"
},
"stat":"identity",
"data":{
"middle":[69.0,414.0,874.0],
"min":[5.0,4.0,8.0],
"max":[398.0,1900.0,4658.0],
"lower":[26.0,139.0,448.0],
"upper":[175.5,849.0,2160.0],
"x":["SAT","UNSAT","UNKNOWN"]
},
"sampling":"none",
"position":"identity",
"geom":"boxplot"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNKNOWN","UNKNOWN","UNKNOWN","UNKNOWN","UNKNOWN","UNKNOWN","UNKNOWN","UNKNOWN","UNKNOWN","UNK

In [17]:
fullSmtData.plot {
    boxplot(result, numberOfVariables)
    y {
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="VFQfAj"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"result",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"numberOfVariables",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"y":"y"
},
"stat":"identity",
"data":{
"min":[2.0,0.0,5.0],
"middle":[28.5,154.0,341.0],
"max":[142.0,625.0,2117.0],
"lower":[10.0,53.0,196.0],
"upper":[63.0,282.0,965.0],
"x":["SAT","UNSAT","UNKNOWN"]
},
"sampling":"none",
"position":"identity",
"geom":"boxplot"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","SAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT","UNSAT

In [27]:
fullSmtData
    .filter { ucNumberOfVariables < 20 }
    .plot {
        boxplot(project, ucNumberOfClauses.mapIndexed { i, n -> numberOfClauses[i].toLong() * n }.rename("value"))
        y {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="6FptzQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"project",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"value",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null],
"trans":"LOG2"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"y":"y"
},
"stat":"identity",
"data":{
"min":[0.0,0.0,0.0,0.0],
"middle":[0.0,0.0,255.0,2484.0],
"max":[1820.0,1200.0,4272.0,9870.0],
"upper":[730.0,480.0,1816.0,4245.0],
"lower":[0.0,0.0,0.0,0.0],
"x":["FASTJSON","GUAVA","SEATA","SPOON"]
},
"sampling":"none",
"position":"identity",
"geom":"boxplot"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FASTJSON","FAST